In [3]:
import pandas as pd

metadata=pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [4]:
import pandas as pd
import os
import librosa
import numpy as np
import librosa.display

def extract_features(file_name):
    data,sample_rate=librosa.load(file_name)
    mfccs = librosa.feature.mfcc(y=data,sr=sample_rate,n_mfcc=40)
    pad_width = 174 - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfccs


In [5]:
from tqdm import tqdm
audio_data=[]
for i,row in tqdm(metadata.iterrows()):
    file_name = f'UrbanSound8K/audio/fold{row["fold"]}/{row["slice_file_name"]}'
    features=extract_features(file_name)
    class_name=row["class"]
    audio_data.append([features,class_name])

featuresdf = pd.DataFrame(audio_data, columns=['feature','class_label'])
featuresdf.head()

3555it [07:59,  7.18it/s]C:\Users\NITHISH\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1323
  return f(*args, **kwargs)
8326it [18:51, 10.21it/s]C:\Users\NITHISH\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1103
  return f(*args, **kwargs)
8328it [18:52, 11.88it/s]C:\Users\NITHISH\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1523
  return f(*args, **kwargs)
8732it [19:44,  7.37it/s]


,feature,class_label
0,"[[-333.38168, -172.86313, -94.09414, -60.86848...",dog_bark
1,"[[-489.11716, -451.05603, -445.70062, -440.020...",children_playing
2,"[[-494.69894, -468.22916, -476.6907, -482.8309...",children_playing
3,"[[-444.25037, -408.51923, -404.58514, -404.187...",children_playing
4,"[[-470.15686, -434.04797, -438.31635, -437.627...",children_playing


In [6]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [7]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
%store x_train 
%store x_test 
%store y_train 
%store y_test 
%store yy 
%store le

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 174
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) 

In [9]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 86, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 41, 64)         8

In [10]:
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

55/55 [==============================] - 2s 21ms/step - loss: 15.8002 - accuracy: 0.0927: 0s - loss: 15.8993 - ac
Pre-training accuracy: 9.2730%


In [11]:
from datetime import datetime 

model.fit(x_train, y_train, batch_size=256, epochs=70, validation_data=(x_test, y_test), verbose=1)


Epoch 1/100
28/28 [==============================] - 25s 861ms/step - loss: 7.8874 - accuracy: 0.1462 - val_loss: 2.5033 - val_accuracy: 0.2290
Epoch 2/100
28/28 [==============================] - 25s 910ms/step - loss: 2.3503 - accuracy: 0.2712 - val_loss: 1.8719 - val_accuracy: 0.3589
Epoch 3/100
28/28 [==============================] - 24s 875ms/step - loss: 1.8422 - accuracy: 0.3752 - val_loss: 1.6606 - val_accuracy: 0.4596
Epoch 4/100
28/28 [==============================] - 24s 852ms/step - loss: 1.5979 - accuracy: 0.4402 - val_loss: 1.5573 - val_accuracy: 0.5009
Epoch 5/100
28/28 [==============================] - 25s 902ms/step - loss: 1.4534 - accuracy: 0.4862 - val_loss: 1.4635 - val_accuracy: 0.5169
Epoch 6/100
28/28 [==============================] - 25s 904ms/step - loss: 1.3617 - accuracy: 0.5251 - val_loss: 1.3934 - val_accuracy: 0.5392
Epoch 7/100
28/28 [==============================] - 25s 880ms/step - loss: 1.2907 - accuracy: 0.5477 - val_loss: 1.3396 - val_accuracy:

In [12]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9322834610939026
Testing Accuracy:  0.8746422529220581


In [93]:
def print_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = np.argmax(model.predict(prediction_feature))
    predicted_class = metadata[metadata["classID"]==predicted_vector]["class"]
    print("The predicted class is:", predicted_class.iloc[0], '\n') 

In [102]:
filename = 'UrbanSound8K/audio/fold1/157867-8-0-17.wav' 
print_prediction(filename)

The predicted class is: siren 

